# [GALAXY] - GAIA Download

<div class="alert alert-block alert-info">
    <b>Note:</b> This notebook should be run with the <span style="font-family: 'Ariel', monospace;">stenv</span> environment
    where the <span style="font-family: 'Ariel', monospace;">regions</span> package has been added if not in the current
    version of <span style="font-family: 'Ariel', monospace;">stenv</span>.
</div>

## Imports

In [ ]:
# Python Imports
import os
from pathlib import Path
from glob import iglob

# Astropy Collab Imports
from astropy import units as u
from astropy.io import fits
from astropy.table import Table, QTable
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from astroquery.gaia import Gaia
from regions import PointSkyRegion, Regions

## Notebook Setup

In [ ]:
# Get the Notebook's Directory and ChangeDir to it
NOTEBOOK_DIR = Path(__file__).resolve().parent
os.chdir(NOTEBOOK_DIR)

## Load External Data

### Load Galaxy Data

In [ ]:
# Get the NED Data
nedDataTable = Table.read('../NED/[GALAXY_SHORT]-NED_Data.ecsv')

In [ ]:
# Get the Coordinates
galCrd = SkyCoord(
    ra=nedDataTable['RA'][0],
    dec=nedDataTable['DEC'][0],
    unit='deg',
    frame='fk5'
)

### Load Images and get Frames

In [ ]:
# Load Images
frames = []
for fn in iglob('../../Images/mastDownload/HST/**/*flc.fits'):

    # Open file for Reading
    with fits.open(fn) as hduList:

        frame = WCS(hduList['SCI', 1], hduList).to_header()['RADESYS']
        if frame not in frames:
            frames.append(frame)

print(frames)

## Get the GAIA Data

In [ ]:
# Get the Object Table
# Note that GAIA aligns to ICRS
Gaia.ROW_LIMIT = -1  # Turn off the GAIA Row Limit
objTable = Gaia.query_object_async(galCrd.icrs, radius=2*u.arcmin)

# Print Number of Sources Found
print(f'Found {len(objTable):d} Objects')

### Write Output to Various Formats

In [ ]:
# Write out the Full Table
objTable.write('[GALAXY_SHORT]-GAIA-AlignmentStars.ecsv', overwrite=True)

In [ ]:
# Get the Coordinates for Writing
# SkyCoord Defaults to ICRS
objCoords = SkyCoord(
    ra=objTable['ra'],
    dec=objTable['dec']
)

# Write Coordinates to File
# Unpack with
# QTable.read('[GALAXY_SHORT]-GAIA-AlignmentStars-Coordinates.ecsv')['SkyCoord']
QTable([objCoords], names=['SkyCoord']).write('[GALAXY_SHORT]-GAIA-AlignmentStars-Coordinates.ecsv', overwrite=True)

In [ ]:
# Get as DS9 Regions
regs = Regions([])
for crd in objCoords:
    regs.append(PointSkyRegion(crd))

# Write to DS9 Region File
regs.write('[GALAXY_SHORT]-GAIA-AlignmentStars-Coordinates.reg', overwrite=True)

# Change Display Style to 'x'
with open('[GALAXY_SHORT]-GAIA-AlignmentStars-Coordinates.reg', 'r') as fid:

    # Get all the Lines
    lines = fid.readlines()

# Add the New Line
lines.insert(1, 'global point=x color=cyan\n')

with open('[GALAXY_SHORT]-GAIA-AlignmentStars-Coordinates.reg', 'w') as fid:

    # Write all the Lines
    fid.writelines(lines)

In [ ]:
# Write as RA, Dec Catalog for TweakReg
with open('[GALAXY_SHORT]-GAIA-RefCatalog-icrs.txt', 'w') as fid:

    # Loop Through Sources
    for crd in objCoords:

        # From TweakReg
        # 'Catalog files themselves must be text files containing "white space"-separated list of values'
        fid.write(f'{crd.icrs.ra.value:<20.8f} {crd.icrs.dec.value:<20.8f}\n')